In [21]:
import requests as req
import bs4
import functools
import tqdm

In [6]:
base_url = "https://api.wikimedia.org/core/v1/wikipedia"
lang = "en"
page_api_base = f"{base_url}/{lang}/page"

In [9]:
html =  req.get("https://en.wikipedia.org/wiki/Bias-variance_tradeoff", allow_redirects=True).content
soup = bs4.BeautifulSoup(html)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Bias–variance tradeoff - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-

In [14]:
list(map(lambda paragraph : paragraph.get_text(), soup.findAll("p")))[-20:]

['Since all three terms are non-negative, the irreducible error forms a lower bound on the expected error on unseen samples.[7]:\u200a34\u200a\n',
 'The more complex the model \n\n\n\n\n\n\nf\n^\n\n\n\n(\nx\n)\n\n\n{\\displaystyle {\\hat {f}}(x)}\n\n is, the more data points it will capture, and the lower the bias will be. However, complexity will make the model "move" more to capture the data points, and hence its variance will be larger.\n',
 'The derivation of the bias–variance decomposition for squared error proceeds as follows.[9][10] For notational convenience, we abbreviate \n\n\n\nf\n=\nf\n(\nx\n)\n\n\n{\\displaystyle f=f(x)}\n\n, \n\n\n\n\n\n\nf\n^\n\n\n\n=\n\n\n\nf\n^\n\n\n\n(\nx\n;\nD\n)\n\n\n{\\displaystyle {\\hat {f}}={\\hat {f}}(x;D)}\n\n and we drop the \n\n\n\nD\n\n\n{\\displaystyle D}\n\n subscript on our expectation operators.\n',
 'Let us write the mean-squared error of our model:\n',
 'First,\n',
 'Secondly, since we model \n\n\n\ny\n=\nf\n+\nε\n\n\n{\\displaystyle 

In [44]:
%%time
semantic_field_katar = [
    "katar",
]

articles = []
for i in semantic_field_katar:
    url = f"https://api.wikimedia.org/core/v1/wikipedia/en/search/page?q={i}&limit=100"
    res: req.Response = req.get(url)
    print(url, res)
    res_dict = res.json()
    pages = res_dict["pages"]
    titles = map(lambda page: page["title"], pages)
    for title in titles:
        resp = req.get(f"https://en.wikipedia.org/wiki/{title}")
        print(title, resp)
        content = resp.content
        soup = bs4.BeautifulSoup(content)
        paragraph_iter = map(lambda tag: tag.get_text(), soup.find_all("p"))
        text = functools.reduce(lambda x, y: x + y, paragraph_iter)
        articles.append((title, text))

print(len(articles))

https://api.wikimedia.org/core/v1/wikipedia/en/search/page?q=katar&limit=100 <Response [200]>
Katar <Response [200]>
Katar (dagger) <Response [200]>
Hawkman (Katar Hol) <Response [200]>
Hawkman <Response [200]>
Joacine Katar Moreira <Response [200]>
Katarism <Response [200]>
Katar Khadak <Response [200]>
Qatar <Response [200]>
Katar, India <Response [200]>
Katar River <Response [200]>
Hawkgirl <Response [200]>
Katar, Bhojpur <Response [200]>
Khooni Katar <Response [200]>
The Chain of Chance <Response [200]>
Shayera Hol <Response [200]>
Hawkwoman <Response [200]>
2022 FIFA World Cup <Response [200]>
John Travolta <Response [200]>
Qatar men's national handball team <Response [200]>
Brianna Hildebrand <Response [200]>
Catarrh <Response [200]>
Hawkman (Carter Hall) <Response [200]>
Push dagger <Response [200]>
List of daggers <Response [200]>
11th Indian Infantry Division <Response [200]>
Hungarian shield <Response [200]>
Mohammad Javad Hosseinnejad <Response [200]>
Georgina Rodríguez <Res

In [49]:
from joblib import Parallel, delayed

def get_article(title):
    resp = req.get(f"https://en.wikipedia.org/wiki/{title}")
    print(title, resp)
    content = resp.content
    soup = bs4.BeautifulSoup(content)
    paragraph_iter = map(lambda tag: tag.get_text(), soup.find_all("p"))
    text = functools.reduce(lambda x, y: x + y, paragraph_iter)
    return title, text
def get_all_articles(titles):
    result_generator = Parallel(n_jobs=4)(delayed(get_article)(title) for title in titles)
    return result_generator


In [50]:
%%time
semantic_field_katar = [
    "katar",
]

articles = []
for i in semantic_field_katar:
    url = f"https://api.wikimedia.org/core/v1/wikipedia/en/search/page?q={i}&limit=100"
    res: req.Response = req.get(url)
    print(url, res)
    res_dict = res.json()
    pages = res_dict["pages"]
    titles = map(lambda page: page["title"], pages)
    articles.extend(get_all_articles(titles))
print(len(articles))

https://api.wikimedia.org/core/v1/wikipedia/en/search/page?q=katar&limit=100 <Response [200]>
Katar <Response [200]>
Hawkman <Response [200]>
Katar (dagger) <Response [200]>
Hawkman (Katar Hol) <Response [200]>
Joacine Katar Moreira <Response [200]>
Katarism <Response [200]>
Katar, India <Response [200]>
Qatar <Response [200]>
Katar River <Response [200]>
Hawkgirl <Response [200]>
Katar, Bhojpur <Response [200]>
The Chain of Chance <Response [200]>
Shayera Hol <Response [200]>
Katar Khadak <Response [200]>
Khooni Katar <Response [200]>
2022 FIFA World Cup <Response [200]>
John Travolta <Response [200]>
Hawkwoman <Response [200]>
Qatar men's national handball team <Response [200]>
Brianna Hildebrand <Response [200]>
Catarrh <Response [200]>
Push dagger <Response [200]>
List of daggers <Response [200]>
11th Indian Infantry Division <Response [200]>
Georgina Rodríguez <Response [200]>
Mohammad Javad Hosseinnejad <Response [200]>
Qatar (disambiguation) <Response [200]>
Rishtey (TV series) 

In [51]:
articles

[('Katar', 'Katar may refer to:\n'),
 ('Katar (dagger)',
  'The katar is a type of push dagger from the Indian subcontinent.[1] The weapon is characterized by its H-shaped horizontal hand grip which results in the blade sitting above the user\'s knuckles. Unique to the Indian subcontinent, it is the most famous and characteristic of Indian daggers.[2] Ceremonial katars were also used in worship.[3]\nHaving originated in South India, the weapon\'s earliest name-form was likely the Tamil kaṭṭāri (கட்டாரி). It is alternatively known in Tamil as kuttuvāḷ (குத்துவாள்) which means "stabbing blade". This was adapted into Sanskrit as kaṭār (कट्टार) or kaṭārī.\nOther regional names for the weapon include kaṭhāri (ಕಠಾರಿ) in Kannada, kathari (కఠారి) in Telugu, kaṭāra (കട്ടാര) in Malayalam, kaṭyāra (कट्यार) in Marathi, kāṭār, (ਕਟਾਰ) in Panjabi, and kirat Dewan language|(chhuiket) in nepalkaṭāra (कटार) or kaṭāri in Hindi.[citation needed]\nThe katar was created in Southern India,[4] its earliest fo

In [120]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")
browser = webdriver.Chrome(options=options)



In [114]:
%%time
browser.get("https://www.aljazeera.com/search/Katar,%20Football%20after:2020")
soup1 = bs4.BeautifulSoup(browser.page_source)


CPU times: user 23.7 ms, sys: 3.95 ms, total: 27.6 ms
Wall time: 1.69 s


In [110]:
%%time
soup2 = bs4.BeautifulSoup(req.get("https://www.aljazeera.com/search/Katar,%20Football").content)

CPU times: user 25.3 ms, sys: 7.87 ms, total: 33.1 ms
Wall time: 124 ms


In [142]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait

browser.get("https://www.aljazeera.com/search/Katar,%20Football%20after:2020")
more_button_found = True

def are_background_activities_complete(driver):
    return driver.execute_script("return (window.fetch && fetch.mostRecentNetworkRequest) ? false : true")

def are_dom_updates_complete(driver):
    return driver.execute_script("return (document.readyState === 'complete' && jQuery.active === 0);")



while more_button_found:
    try:
        more_button = browser.find_element(By.CLASS_NAME, ("show-more-button"))
        browser.execute_script("arguments[0].click();", more_button)
        WebDriverWait(browser, 5).until(
        are_background_activities_complete
        )
        WebDriverWait(browser).until(
            are_dom_updates_complete
        )
    except NoSuchElementException:
        more_button_found = False


In [143]:
full_result_soup = bs4.BeautifulSoup(browser.page_source)

links = full_result_soup.find("div", class_="search-result__list").find_all("a")
for i in links:
    print(i["href"])

https://www.aljazeera.com/sports/2022/10/17/qatar-to-host-2023-afc-asian-cup
https://www.aljazeera.com/sports/2023/6/21/qatar-backs-footballer-as-racism-row-continues
https://www.aljazeera.com/news/2023/7/3/qatar-stun-mexico-to-reach-quarter-finals-of-gold-cup-football
https://www.aljazeera.com/sports/2022/10/9/timeline-a-history-of-qatar-football
https://www.aljazeera.com/sports/2022/11/20/fifa-revenue-hits-7-5b-for-current-world-cup-period
https://www.aljazeera.com/news/2023/12/16/stand-with-palestine-football-match-charity-no-place-for-genocide-israel-war-gaza-palestinians-children
https://www.aljazeera.com/qatar-world-cup-2022/
https://www.aljazeera.com/opinions/2022/12/7/fifa-world-cup-palestine-1-israel-0
https://www.aljazeera.com/sports/2023/12/6/qatar-football-team-qfa-coach-carlos-queiroz-terminate-contract-mutual-agreement-asian-cup-2023-marques-lopez-hired
https://www.aljazeera.com/sports/2022/7/29/where-will-the-teams-stay-train-at-qatar-world-cup-2022


In [95]:
len(links)

10

In [69]:
len(soup1.findAll("a"))

65

In [75]:
import json
import urllib.parse

jazeera_graph_ql_backend = "https://www.aljazeera.com/graphql"

jazeera_search_vars = {
    "query": "katar football after:2020",
    "start": 1,
    "sort": "relevance"
}


jazeera_search_vars_str = json.dumps(jazeera_search_vars)
jazeera_search_vars_str = urllib.parse.quote(jazeera_search_vars_str)

jazeera_url = f"https://www.aljazeera.com/graphql?wp-site=aje&operationName=SearchQuery&variables={jazeera_search_vars_str}&extensions=%7B%7D"

res = req.get(jazeera_url)
res.content


b'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;aljazeera&#46;com&#47;graphql&#63;" on this server.<P>\nReference&#32;&#35;18&#46;476656b8&#46;1705155692&#46;168ceca7\n</BODY>\n</HTML>\n'